In [1]:
import logging

import numpy

from cicada import transcript
from cicada.arithmetic import Field

In [2]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype=object))
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 88805302054355216233154397747606998133, 'inc': 273519481568979284389875895496234343711}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([149, 142, 225], dtype=object))
lhs = numpy.array([1, 1, 1], dtype=object)
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([149, 142, 225], dtype=object))
cicada.transcript.assert_equal(lhs, numpy.array([23, 16, 99], dtype=object))


In [3]:
from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    transcript.log("Setup additive sharing")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Share some secrets")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Add some secrets")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Reveal the results")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=2);

# Player 1: Setup additive sharing
# Player 0: Setup additive sharing
# Player 1: Share some secrets
# Player 0: Share some secrets
bg = numpy.random.PCG64()
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 329900914738071519479702908717079448344, 'inc': 155367840184836092189066130057971594069}, 'has_uint32': 0, 'uinteger': 0}
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 100673059414612779622537062608723047634, 'inc': 158771522073355783319438860632892046061}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(10, dtype=object))
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(204, dtype=object))
bg = numpy.random.PCG64()
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 329900914738071519479702908717079448344, '

In [4]:
def main(communicator):
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler())
    
    transcript.log("Setup additive sharing")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Share some secrets")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Add some secrets")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Reveal the results")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=2);

Player 0: Setup additive sharing
Player 1: Setup additive sharing
Player 1: --> 0 PRZS 307003437199772683
Player 0: --> 1 PRZS 7239575226077358250
Player 1: <-- 0 PRZS 7239575226077358250
Player 0: <-- 1 PRZS 307003437199772683
Player 1: Share some secrets
Player 0: Share some secrets
Player 1: Add some secrets
Player 0: Add some secrets
Player 0: Reveal the results
Player 1: Reveal the results
Player 0: --> 0 GATHER 22
Player 1: --> 0 GATHER 125
Player 0: <-- 0 GATHER 22
Player 1: --> 1 GATHER 125
Player 0: <-- 1 GATHER 125
Player 1: <-- 1 GATHER 125
Player 0: --> 1 GATHER 22
Player 1: <-- 0 GATHER 22
